In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import missingno as msno
import seaborn as sns; sns.set(style='ticks', context='talk')
from sklearn import preprocessing
import cartopy
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import glob
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geopandas as gpd
import itertools

mapbox = cimgt.MapboxTiles('pk.eyJ1IjoiamVzc2VwaXNlbCIsImEiOiJjamdmaHlxajkybnhmMnFtcXppaW90bDVjIn0.i2qzLErAxTjsmwl127P-jQ', 'mapbox.high-contrast')

In [2]:
hucs = gpd.read_file(r'J:\Geology\WSGS\GIS\huc_12.shp')
hucs = hucs.to_crs(crs={'init': 'epsg:3732'})

In [3]:
weights = pd.DataFrame()
a=hucs.HUC_12.unique()[0:10]
c = list(itertools.product(a, a))
weights['HUC12'] = list(zip(*c))[0]
weights['to'] = list(zip(*c))[1]

In [11]:
hucs['HUC_12'].head()

0    100700060606
1    100700010708
2    100800100502
3    100800140404
4    101102010303
Name: HUC_12, dtype: object

In [14]:
hucs['HU_12_DS'].head()

0    100700060703
1    100700010707
2    100800100503
3    100800140406
4    101102010306
Name: HU_12_DS, dtype: object